In [1]:
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.ml.feature import Bucketizer
import pyspark.sql.functions as f

In [2]:
sc = SparkContext()

In [3]:
spark = SQLContext(sc)

In [4]:
from pyspark.sql.types import *
newDF=[StructField('ticker',StringType(),True),
       StructField('date',StringType(),True),
       StructField('time',StringType(),True),
       StructField('open',FloatType(),True),
       StructField('high',FloatType(),True),
       StructField('low',FloatType(),True),
       StructField('close',FloatType(),True),
       StructField('vol',IntegerType(),True)
       ]
finalStruct=StructType(fields=newDF)

In [5]:
df = spark.read.csv('/home/indranil/prices/all_csv/*.csv', schema=finalStruct,header=False)

In [6]:
df.printSchema()

root
 |-- ticker: string (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- vol: integer (nullable = true)



In [7]:
df.show()

+------+--------+------+------+------+------+------+----+
|ticker|    date|  time|  open|  high|   low| close| vol|
+------+--------+------+------+------+------+------+----+
|  null|    null|  null|  null|  null|  null|  null|null|
|GBPJPY|20010102|230100|171.88|171.88|171.86|171.86|   4|
|GBPJPY|20010102|230200|171.86|171.87|171.86|171.87|   4|
|GBPJPY|20010102|230300|171.87|171.87|171.87|171.87|   4|
|GBPJPY|20010102|230400|171.87|171.88|171.87|171.88|   4|
|GBPJPY|20010102|230500|171.88|171.88|171.88|171.88|   4|
|GBPJPY|20010102|230600|171.88|171.88|171.88|171.88|   4|
|GBPJPY|20010102|230700|171.88|171.88|171.88|171.88|   4|
|GBPJPY|20010102|230800|171.88|171.88|171.87|171.87|   4|
|GBPJPY|20010102|230900|171.87|171.87|171.87|171.87|   4|
|GBPJPY|20010102|231100|171.87|171.88|171.87|171.88|   4|
|GBPJPY|20010102|231200|171.88|171.89|171.88|171.89|   4|
|GBPJPY|20010102|231300|171.89|171.89|171.89|171.89|   4|
|GBPJPY|20010102|231400|171.88|171.88|171.81|171.81|   4|
|GBPJPY|200101

In [11]:
df = df.filter(~df['ticker'].isin(['<TICKER>']))

In [12]:
df.groupBy('ticker').count().show()

+------+-------+
|ticker|  count|
+------+-------+
|AUDUSD|6390609|
|AUDJPY|6570214|
|XAUUSD|4812877|
|NZDUSD|5946635|
|GBPUSD|6601900|
|XAGUSD|3492265|
|GBPCHF|6758590|
|USDJPY|6655187|
|EURCAD|6653964|
|EURGBP|6549767|
|CHFJPY|6659533|
|EURJPY|6710031|
|USDCHF|6700279|
|GBPJPY|6741260|
|NZDJPY|6025340|
|EURCHF|6701551|
|USDCAD|6392282|
+------+-------+



In [14]:
df.dtypes

[('ticker', 'string'),
 ('date', 'string'),
 ('time', 'string'),
 ('open', 'float'),
 ('high', 'float'),
 ('low', 'float'),
 ('close', 'float'),
 ('vol', 'int')]

In [15]:
from pyspark.sql.functions import udf, struct

In [16]:
test_str = "20010102"

In [17]:
test_str[-2:]

'02'

In [18]:
get_year = udf(lambda x:x[:4],StringType())

In [19]:
get_month = udf(lambda x:x[4:6],StringType())

In [20]:
get_day = udf(lambda x:x[-2:],StringType())

In [21]:
get_hour = udf(lambda x:x[:2],StringType())

In [22]:
get_min = udf(lambda x:x[2:4],StringType())

In [26]:
df = df.withColumn("year", get_year(df['date']))

In [27]:
df = df.withColumn("month", get_month(df['date']))

In [28]:
df = df.withColumn("day", get_day(df['date']))

In [29]:
df = df.withColumn("hour", get_hour(df['time']))

In [30]:
df = df.withColumn("minute", get_min(df['time']))

In [31]:
df.cube(['month','day','hour','minute']).count().count()

625250

In [32]:
import numpy as np 
from pyspark.ml.feature import Bucketizer

In [40]:
def approx_quartile(df,col,outcol,qur =8):
    y = list(np.true_divide(np.arange(qur),qur-1))
    y.pop(0)
    splits = df.approxQuantile(col, y, 0.0005) # no need
    splits.insert(0,0)
    bucketizer = Bucketizer(splits=splits,inputCol=col, outputCol=outcol)
    df= bucketizer.transform(df)
    return df
    


In [46]:
df = approx_quartile(df,"high","buckethigh")

In [47]:
df = approx_quartile(df,"low","bucketlow")

In [49]:
df = approx_quartile(df,"open","bucketopen")

In [50]:
df = approx_quartile(df,"close","bucketclose")

In [51]:
df.columns

['ticker',
 'date',
 'time',
 'open',
 'high',
 'low',
 'close',
 'vol',
 'year',
 'month',
 'day',
 'hour',
 'minute',
 'bucket_high',
 'bucket_low',
 'buckethigh',
 'bucketlow',
 'bucketopen',
 'bucketclose']

In [52]:
df_high = df.filter(df['bucketopen'].isin([0.0])).filter(df['bucketclose'].isin([8.0]))

In [54]:
df.groupBy('bucketopen').count().show()

+----------+--------+
|bucketopen|   count|
+----------+--------+
|       0.0|15229401|
|       1.0|15170848|
|       4.0|15140261|
|       3.0|15225579|
|       2.0|15220834|
|       6.0|15171629|
|       5.0|15203732|
+----------+--------+



In [36]:
sum_v =df1.agg(f.sum('close'))

In [37]:
sum_v.show()

+-------------------+
|         sum(close)|
+-------------------+
|5.525987008346418E8|
+-------------------+

